In [92]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import datetime

weekday = { 0:'MON',
            1:'TUE',
            2:'WED',
            3:'THU',
            4:'FRI',
            5:'SAT',
            6:'SUN'
        }

year_list = [2023]
month_list = [9]
day_list = [i for i in range(1,32)]
for year in year_list:
    for month in month_list:
        init_datetime = datetime.datetime(year, month, 1)
        # Get the year, week number, and weekday using isocalendar()
        year, init_week_number, _ = init_datetime.isocalendar()
        for day in day_list:
            specific_datetime = datetime.datetime(year,month,day)
            year, specific_week_number, _ = specific_datetime.isocalendar()
            
            specific_week_number = init_week_number-specific_week_number + 1
            # Chrome 브라우저를 사용하여 웹 드라이버 시작
            driver = webdriver.Chrome()

            # Agoda 사이트로 이동
            driver.get("https://www.agoda.com/ko-kr/")

            # 페이지가 로딩될 때까지 잠시 기다립니다.
            time.sleep(10)

            # 팝업창 클릭해서 없애기
            popup_box = driver.find_element(By.XPATH,"/html/body/div[12]/div[2]/button")
            popup_box.click()

            time.sleep(4)

            # 검색어 입력란을 찾고 "로스엔젤레스"라는 검색어를 입력합니다.

            search_box = driver.find_element(By.XPATH,'//*[@id="textInput"]')
            search_box.send_keys("로스엔젤레스")
            search_box.send_keys(Keys.RETURN)

            time.sleep(4)
            autocomplete_tab = driver.find_element(By.XPATH,'//*[@id="SearchBoxContainer"]/div[1]/div/div[2]/div/div/div[6]/div/div/ul/li[1]')
            autocomplete_tab.click()
            time.sleep(4)

            # 달력에서 다음 버튼 클릭
            next_month_button = driver.find_element(By.XPATH,'//*[@id="SearchBoxContainer"]/div[1]/div/div[2]/div/div/div[6]/div/div/div[1]/div/div[1]/span[2]')
            next_month_button.click()
            time.sleep(4)

            # 날짜 선택 및 클릭
            start_date_picker = driver.find_element(By.XPATH,'//*[@id="SearchBoxContainer"]/div[1]/div/div[2]/div/div/div[6]/div/div/div[1]/div/div[2]/div[2]/div[3]/div[1]/div[5]')
            start_date_picker.click()
            time.sleep(1)
            end_date_picker = driver.find_element(By.XPATH,'//*[@id="SearchBoxContainer"]/div[1]/div/div[2]/div/div/div[6]/div/div/div[1]/div/div[2]/div[2]/div[3]/div[1]/div[6]')
            end_date_picker.click()
            time.sleep(1)
            minus_picker = driver.find_element(By.XPATH,'//*[@id="occupancy-selector"]/div/div/div[2]/div[2]/div[1]')
            minus_picker.click()
            time.sleep(1)
            search_button = driver.find_element(By.XPATH,'//*[@id="SearchBoxContainer"]/div[2]/button')
            search_button.click()
            time.sleep(3)
            search_button = driver.find_element(By.XPATH,'//*[@id="SearchBoxContainer"]/div[2]/button')
            search_button.click()
            time.sleep(3)

# search_box.send_keys(Keys.RETURN)

# 검색 결과가 로딩될 때까지 잠시 기다립니다.
time.sleep(10)


# 숙소 위치 평가 선택
value9_filter = driver.find_element(By.XPATH,'//*[@id="SideBarLocationFilters"]/div[10]/div[2]/ul/li[1]/span')
value9_filter.click()
time.sleep(7)

# 투숙객 평가 점수 선택
customer_eval_filter = driver.find_element(By.XPATH,'//*[@id="SideBarLocationFilters"]/div[8]/div[2]/ul/li[1]/span')
customer_eval_filter.click()
time.sleep(7)

# 호텔 리스트 추출
# hotel_list_containers = driver.find_elements(By.XPATH, '//div[@class="hotel-list-container"]')
hotel_elements = driver.find_elements(By.XPATH, '//*[@class="PropertyCard PropertyCardItem"]')

# 웹 브라우저를 닫습니다.
# driver.quit()

In [49]:
# CREATE Accommodation (
#   ID INT
#   시설 이름 CHAR(100)
#   별점 INT
#   시설 유형 CHAR(100)
#   가격 (INT)
#   위치 CHAR(100)
#   체크인 날짜 DATETIME
#   요일 CHAR(100)
# );

## 

In [97]:
import datetime

hotel_elements[0].get_property('outerHTML')

hotel_name = hotel_elements[0].find_elements(By.TAG_NAME,'h3')[0].text
hotel_location = hotel_elements[0].find_elements(By.TAG_NAME,'span')[5].text
hotel_price = hotel_elements[0].find_elements(By.CLASS_NAME,'PropertyCardPrice__Value')[0].text
len(hotel_elements)


Current date: 2023-07-21
Current weekday (0: Monday, 6: Sunday): 4


In [1]:
str1 = '//*[@id="SearchBoxContainer"]/div[1]/div/div[2]/div/div/div[6]/div/div/div[1]/div/div[2]/div[2]/div[3]/div[1]/div[6]'
str2 = '//*[@id="SearchBoxContainer"]/div[1]/div/div[2]/div/div/div[6]/div/div/div[1]/div/div[2]/div[2]/div[3]/div[1]/div[7]'
str1 == str2

str1 = '//*[@id="SearchBoxContainer"]/div[1]/div/div[2]/div/div/div[6]/div/div/div[1]/div/div[2]/div[1]/div[3]/div[1]/div[5]' # FRI
str2 = '//*[@id="SearchBoxContainer"]/div[1]/div/div[2]/div/div/div[6]/div/div/div[1]/div/div[2]/div[1]/div[3]/div[1]/div[6]' # SAT 
str3 = '//*[@id="SearchBoxContainer"]/div[1]/div/div[2]/div/div/div[6]/div/div/div[1]/div/div[2]/div[1]/div[3]/div[1]/div[7]' # SUN
str4 = '//*[@id="SearchBoxContainer"]/div[1]/div/div[2]/div/div/div[6]/div/div/div[1]/div/div[2]/div[1]/div[3]/div[2]/div[1]'
str30 = '//*[@id="SearchBoxContainer"]/div[1]/div/div[2]/div/div/div[6]/div/div/div[1]/div/div[2]/div[1]/div[3]/div[5]/div[6]'

In [10]:
import datetime

# Define the specific datetime object
year = 2023
month = 9
day = 1
specific_datetime = datetime.datetime(year, month, day)

# Get the year, week number, and weekday using isocalendar()
year, week_number, _ = specific_datetime.isocalendar()

# Print the week number
print("Week number in the month:", week_number)

Week number in the month: 35
